In [53]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import StratifiedShuffleSplit

In [11]:
datapath = r'D:/projects/congress/processed/training_vectors/'

In [30]:
true_switchers_ = ['A000361','B000229', 'B001264', 'D000168', 'G000280', 'H000067', 'H000390', 'L000119', 'P000066', 'T000058', 'C000077','F000257', 'G000557', 
'S000320', 'S000709','J000072']

independents = ['S000033', 'B001237', 'K000383']

true_switchers =independents + true_switchers_
enddate = '2020-10-04'

In [23]:
dws = np.load(datapath + 'dws_2009_20.npy')
dws.shape

(208292,)

In [66]:
As_all = np.load(datapath+ 'author_dummy_2009_20.npy')
As_all.shape

(208292, 1674)

In [26]:
congress_df = pd.read_csv(r'D:\projects\congress\processed\processed_reports\crs_2009_20_10_04_trancated.csv', sep ='\t', encoding = 'ISO-8859-1', index_col = 0)

In [27]:
congress_df.shape

(208292, 11)

In [69]:
congress_df['dw'] = dws
congress_df['author'] = list(As_all)

In [70]:
congress_df.head()

,speech_id,true_id,chamber,state,party,congress,daysafter,monthsafter,speech,date,topics_prior,dw,author
594060,1110000007,L000557,H,CT,D,111,6578,216,our democracy renews itself every 2 years as m...,20090106,22,-0.398,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
594061,1110000009,P000587,H,IN,R,111,6578,216,madam clerk. as chairman of the republican con...,20090106,22,0.655,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
594062,1110000013,B000589,H,OH,R,111,6578,216,. leader hoyer. fellow members. and a special ...,20090106,18,0.513,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
594063,1110000014,P000197,H,CA,D,111,6578,216,thank you very much. leader boehner. together....,20090106,22,-0.490,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
594064,1110000016,L000557,H,CT,D,111,6578,216,"for""your nomination this morning. thank you to...",20090106,-1,-0.398,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [71]:
# filter out all switchers
congress_flt_ = congress_df.loc[~congress_df['true_id'].isin(true_switchers)]
congress_flt_['is_Dem'] = (congress_flt_['party'] == 'D')*1

D:\anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [72]:
congress_flt = congress_flt_.sort_values(by = ['dw'], ascending = True, ignore_index = True)

In [73]:
ds_ns = np.load(r'D:\projects\congress\processed\training_vectors\all_berts_vec_noswitcher.npy')

In [74]:
ds_ns.shape

(206899, 768)

In [38]:
congress_flt.head()

,speech_id,true_id,chamber,state,party,congress,daysafter,monthsafter,speech,date,topics_prior,dw,is_Dem
0,1130067801,W000817,S,MA,D,113,8405,276,i am here today with some good news. this week...,20140107,23,-0.769,1
1,1130082890,W000817,S,MA,D,113,8484,278,. this is a difficult day for the city of bost...,20140327,45,-0.769,1
2,1140095053,W000817,S,MA,D,114,9272,304,. 8 years ago reckless bankers on wall street ...,20160523,64,-0.769,1
3,1130074735,W000817,S,MA,D,113,8441,277,. i thank senators durbin and reed for their e...,20140212,18,-0.769,1
4,1130074743,W000817,S,MA,D,113,8441,277,. i would like to speak for another minute abo...,20140212,64,-0.769,1


In [39]:
ids_ns = congress_flt['speech_id'].values

In [44]:
parties_ns = congress_flt['is_Dem'].values
thetas_ns = congress_flt['topics_prior'].values

In [80]:
As_ns = np.array(congress_flt['author'].values.tolist())
As_ns.shape

(206899, 1674)

In [78]:
biased = True
notmask = 0.8

In [81]:
if not biased:
    sss = StratifiedShuffleSplit(n_splits=5, test_size= 1- notmask)
    train_id, test_id= next(sss.split(thetas_ns, parties_ns))
else:
    print('start biased sampling')
    orderid = np.arange(len(parties_ns))
    tbn = int(len(parties_ns) * notmask/2)
    train_id = np.concatenate([orderid[:tbn], orderid[-tbn:]])
    test_id = orderid[tbn:-tbn]
    np.random.shuffle(train_id)
    np.random.shuffle(test_id)
    print(len(train_id), len(test_id))


ids, ids_val = ids_ns[train_id], ids_ns[test_id]
ds, ds_val = ds_ns[train_id], ds_ns[test_id]
# As, As_val = As_ns[train_id], As_ns[test_id]
# thetas0, thetas_val = thetas_ns[train_id], thetas_ns[test_id]
parties, parties_val = parties_ns[train_id], parties_ns[test_id]
As, As_val = As_ns[train_id], As_ns[test_id]


start biased sampling
165518 41381


In [82]:
train_id, test_id

(array([142929,  74266,  21260, ..., 184711, 133379,  54700]),
 array([ 88536, 111616, 101801, ..., 101653,  89281,  99726]))

In [83]:
As.shape

(165518, 1674)